In [370]:
import pymysql 
import pandas as pd

In [371]:
def create_database():
    #connect to the default database
    conn= pymysql.connect(host='localhost',user='root',db='test_db',password='Password')
    cur = conn.cursor()
    
    # Set autocommit to True
    cur.execute("SET autocommit = 1")
    
    #create a sparkify database with UTF8 encoding
    cur.execute("DROP DATABASE IF EXISTS accounts1")
    cur.execute("CREATE DATABASE accounts1")
    
    #close connection to default database
    conn.close()
    
    #connect to sparkify databases
    conn = pymysql.connect(host='localhost',user='root',db='accounts1',password='Password')
    cur = conn.cursor()
    
    return cur, conn


In [372]:
AccountsCountry = pd.read_csv("Wealth-AccountsCountry.csv")

In [373]:
AccountsCountry.head()

,Code,Long Name,Income Group,Region,Lending category,Other groups,Currency Unit,Latest population census,Latest household survey,Special Notes,...,Source of most recent Income and expenditure data,Vital registration complete,Latest agricultural census,Latest industrial data,Latest trade data,Latest water withdrawal data,2-alpha code,WB-2 code,Table Name,Short Name
0,ALB,Republic of Albania,Upper middle income,Europe & Central Asia,IBRD,NaN,Albanian lek,2020 (expected),"Demographic and Health Survey, 2017/18",NaN,...,Living Standards Measurement Study Survey (LSM...,Yes,2012,2013.0,2018.0,2006.0,AL,AL,Albania,Albania
1,ARG,Argentine Republic,Upper middle income,Latin America & Caribbean,IBRD,NaN,Argentine peso,2020 (expected),"Multiple Indicator Cluster Survey, 2019/20",NaN,...,"Integrated household survey (IHS), 2016",Yes,2008,2002.0,2018.0,2011.0,AR,AR,Argentina,Argentina
2,ARM,Republic of Armenia,Upper middle income,Europe & Central Asia,IBRD,NaN,Armenian dram,2020 (expected),"Demographic and Health Survey, 2015/16",NaN,...,"Integrated household survey (IHS), 2016",Yes,2014,NaN,2018.0,2012.0,AM,AM,Armenia,Armenia
3,AUS,Commonwealth of Australia,High income,East Asia & Pacific,NaN,NaN,Australian dollar,2016,NaN,Fiscal year end: June 30; reporting period for...,...,"Expenditure survey/budget survey (ES/BS), 2010",Yes,2015-2016,2013.0,2018.0,2013.0,AU,AU,Australia,Australia
4,AUT,Republic of Austria,High income,Europe & Central Asia,NaN,Euro area,Euro,2011. Population figures compiled from adminis...,NaN,A simple multiplier is used to convert the nat...,...,"Income survey (IS), 2015",Yes,2010,2014.0,2018.0,2010.0,AT,AT,Austria,Austria


In [374]:
AccountsCountry_clean= AccountsCountry[['Code','Long Name','Table Name','Short Name','Currency Unit']]

In [375]:
AccountsCountry_clean.head()

,Code,Long Name,Table Name,Short Name,Currency Unit
0,ALB,Republic of Albania,Albania,Albania,Albanian lek
1,ARG,Argentine Republic,Argentina,Argentina,Argentine peso
2,ARM,Republic of Armenia,Armenia,Armenia,Armenian dram
3,AUS,Commonwealth of Australia,Australia,Australia,Australian dollar
4,AUT,Republic of Austria,Austria,Austria,Euro


In [376]:
AccountsCountry_clean.isnull().sum()

Code                5
Long Name           3
Table Name       7451
Short Name       7451
Currency Unit    7451
dtype: int64

In [377]:
AccountsCountry_clean = AccountsCountry_clean.fillna(0)

In [378]:
AccountsCountry_clean.shape

(7597, 5)

In [379]:
AccountsCountry_clean.columns = AccountsCountry_clean.columns.str.replace('Code', 'Country Code')

In [393]:
AccountsCountry_clean.head()

,Country Code,Long Name,Table Name,Short Name,Currency Unit
0,ALB,Republic of Albania,Albania,Albania,Albanian lek
1,ARG,Argentine Republic,Argentina,Argentina,Argentine peso
2,ARM,Republic of Armenia,Armenia,Armenia,Armenian dram
3,AUS,Commonwealth of Australia,Australia,Australia,Australian dollar
4,AUT,Republic of Austria,Austria,Austria,Euro


In [394]:
AccountsCountry_clean.drop_duplicates(subset=['Country Code'], keep='first', inplace=True)

In [395]:
AccountsCountry_clean.head()

,Country Code,Long Name,Table Name,Short Name,Currency Unit
0,ALB,Republic of Albania,Albania,Albania,Albanian lek
1,ARG,Argentine Republic,Argentina,Argentina,Argentine peso
2,ARM,Republic of Armenia,Armenia,Armenia,Armenian dram
3,AUS,Commonwealth of Australia,Australia,Australia,Australian dollar
4,AUT,Republic of Austria,Austria,Austria,Euro


In [408]:
AccountsData = pd.read_csv("Wealth-AccountData.csv")

In [409]:
AccountsData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7592 entries, 0 to 7591
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Country Name   7592 non-null   object 
 1   Country Code   7592 non-null   object 
 2   Series Name    7592 non-null   object 
 3   Series Code    7592 non-null   object 
 4   1995 [YR1995]  7592 non-null   object 
 5   2000 [YR2000]  7592 non-null   object 
 6   2005 [YR2005]  7592 non-null   float64
 7   2010 [YR2010]  7592 non-null   float64
 8   2015 [YR2015]  7592 non-null   float64
dtypes: float64(3), object(6)
memory usage: 533.9+ KB


In [410]:
AccountsData.head()

,Country Name,Country Code,Series Name,Series Code,1995 [YR1995],2000 [YR2000],2005 [YR2005],2010 [YR2010],2015 [YR2015]
0,Albania,ALB,Human capital (constant 2018 US$),NW.HCA.TO,44900000000.00,43600000000.00,5.440000e+10,6.810000e+10,7.370000e+10
1,Albania,ALB,Human capital per capita (constant 2018 US$),NW.HCA.PC,14072.79,14118.13,1.807230e+04,2.336286e+04,2.559371e+04
2,Albania,ALB,"Human capital per capita, employed female (con...",NW.HCA.FEMP.PC,3468.86,3424.01,2.222120e+03,2.889150e+03,2.987550e+03
3,Albania,ALB,"Human capital per capita, employed male (const...",NW.HCA.MEMP.PC,6507.93,6872.02,1.168935e+04,1.578992e+04,1.730701e+04
4,Albania,ALB,"Human capital per capita, female (constant 201...",NW.HCA.FEMA.PC,4893.01,4695.07,2.886740e+03,3.613610e+03,3.767630e+03


In [411]:
AccountsData.columns

Index(['Country Name', 'Country Code', 'Series Name', 'Series Code',
       '1995 [YR1995]', '2000 [YR2000]', '2005 [YR2005]', '2010 [YR2010]',
       '2015 [YR2015]'],
      dtype='object')

In [412]:
AccountsData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7592 entries, 0 to 7591
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Country Name   7592 non-null   object 
 1   Country Code   7592 non-null   object 
 2   Series Name    7592 non-null   object 
 3   Series Code    7592 non-null   object 
 4   1995 [YR1995]  7592 non-null   object 
 5   2000 [YR2000]  7592 non-null   object 
 6   2005 [YR2005]  7592 non-null   float64
 7   2010 [YR2010]  7592 non-null   float64
 8   2015 [YR2015]  7592 non-null   float64
dtypes: float64(3), object(6)
memory usage: 533.9+ KB


In [413]:
AccountsData = AccountsData[['Country Name', 'Country Code', 'Series Name', 'Series Code','1995 [YR1995]', '2000 [YR2000]','2005 [YR2005]', '2010 [YR2010]','2015 [YR2015]']]

In [414]:
AccountsData = AccountsData.rename(columns={
    '1995 [YR1995]': '1995',
    '2000 [YR2000]': '2000',
    '2005 [YR2005]': '2005',
    '2010 [YR2010]': '2010',
    '2015 [YR2015]': '2015'
})


In [415]:
AccountsData.head()

,Country Name,Country Code,Series Name,Series Code,1995,2000,2005,2010,2015
0,Albania,ALB,Human capital (constant 2018 US$),NW.HCA.TO,44900000000.00,43600000000.00,5.440000e+10,6.810000e+10,7.370000e+10
1,Albania,ALB,Human capital per capita (constant 2018 US$),NW.HCA.PC,14072.79,14118.13,1.807230e+04,2.336286e+04,2.559371e+04
2,Albania,ALB,"Human capital per capita, employed female (con...",NW.HCA.FEMP.PC,3468.86,3424.01,2.222120e+03,2.889150e+03,2.987550e+03
3,Albania,ALB,"Human capital per capita, employed male (const...",NW.HCA.MEMP.PC,6507.93,6872.02,1.168935e+04,1.578992e+04,1.730701e+04
4,Albania,ALB,"Human capital per capita, female (constant 201...",NW.HCA.FEMA.PC,4893.01,4695.07,2.886740e+03,3.613610e+03,3.767630e+03


In [418]:
AccountsData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7592 entries, 0 to 7591
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Country Name  7592 non-null   object 
 1   Country Code  7592 non-null   object 
 2   Series Name   7592 non-null   object 
 3   Series Code   7592 non-null   object 
 4   1995          7582 non-null   float64
 5   2000          7590 non-null   float64
 6   2005          7592 non-null   float64
 7   2010          7592 non-null   float64
 8   2015          7592 non-null   float64
dtypes: float64(5), object(4)
memory usage: 533.9+ KB


In [417]:
AccountsData['1995'] = pd.to_numeric(AccountsData['1995'], errors='coerce')
AccountsData['2000'] = pd.to_numeric(AccountsData['2000'], errors='coerce')

In [419]:
AccountsData[AccountsData.isna().any(axis=1)]

,Country Name,Country Code,Series Name,Series Code,1995,2000,2005,2010,2015
1224,Chad,TCD,"Natural capital per capita, nonrenewable asset...",NW.NCA.SAOI.PC,NaN,NaN,1.451300e+03,3.303950e+03,2.564790e+03
1239,Chad,TCD,"Natural capital, nonrenewable assets: oil (con...",NW.NCA.SAOI.TO,NaN,NaN,1.470000e+10,3.950000e+10,3.620000e+10
6369,Suriname,SUR,"Natural capital per capita, nonrenewable asset...",NW.NCA.SACO.PC,NaN,0.0,0.000000e+00,0.000000e+00,0.000000e+00
6384,Suriname,SUR,"Natural capital, nonrenewable assets: coal (co...",NW.NCA.SACO.TO,NaN,0.0,0.000000e+00,0.000000e+00,0.000000e+00
7409,West Bank and Gaza,PSE,"Natural capital per capita, nonrenewable asset...",NW.NCA.SACO.PC,NaN,0.0,0.000000e+00,0.000000e+00,0.000000e+00
7410,West Bank and Gaza,PSE,"Natural capital per capita, nonrenewable asset...",NW.NCA.SAGA.PC,NaN,0.0,0.000000e+00,0.000000e+00,0.000000e+00
7412,West Bank and Gaza,PSE,"Natural capital per capita, nonrenewable asset...",NW.NCA.SAOI.PC,NaN,0.0,0.000000e+00,0.000000e+00,0.000000e+00
7424,West Bank and Gaza,PSE,"Natural capital, nonrenewable assets: coal (co...",NW.NCA.SACO.TO,NaN,0.0,0.000000e+00,0.000000e+00,0.000000e+00
7425,West Bank and Gaza,PSE,"Natural capital, nonrenewable assets: gas (con...",NW.NCA.SAGA.TO,NaN,0.0,0.000000e+00,0.000000e+00,0.000000e+00
7427,West Bank and Gaza,PSE,"Natural capital, nonrenewable assets: oil (con...",NW.NCA.SAOI.TO,NaN,0.0,0.000000e+00,0.000000e+00,0.000000e+00


In [420]:
AccountsData.fillna(0, inplace=True)

In [421]:
AccountsData[AccountsData.isna().any(axis=1)].isna().any()

Country Name    False
Country Code    False
Series Name     False
Series Code     False
1995            False
2000            False
2005            False
2010            False
2015            False
dtype: bool

In [209]:
AccountsSeries = pd.read_csv("Wealth-AccountSeries.csv")

In [438]:
AccountsSeries.head()

,Series Code,Topic,Indicator Name,Long definition
0,NW.HCA.TO,Human capital,Human capital (constant 2018 US$),Human capital is computed as the present value...
1,NW.HCA.PC,Human capital,Human capital per capita (constant 2018 US$),Human capital is computed as the present value...
2,NW.HCA.FEMP.PC,Human capital,"Human capital per capita, employed female (con...",Human capital is computed as the present value...
3,NW.HCA.MEMP.PC,Human capital,"Human capital per capita, employed male (const...",Human capital is computed as the present value...
4,NW.HCA.FEMA.PC,Human capital,"Human capital per capita, female (constant 201...",Human capital is computed as the present value...


In [448]:
AccountsSeries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Series Code      52 non-null     object
 1   Topic            52 non-null     object
 2   Indicator Name   52 non-null     object
 3   Long definition  52 non-null     object
dtypes: object(4)
memory usage: 1.8+ KB


In [212]:
AccountsSeries.columns = AccountsSeries.columns.str.replace('Code', 'Series Code')

In [437]:
AccountsSeries = AccountsSeries[[ 'Series Code','Topic','Indicator Name','Long definition']]

In [214]:
cur, conn = create_database()

In [290]:
Accounts_Country_table1 = ("DROP TABLE IF EXISTS AccountsCountry")
cur.execute(Accounts_Country_table1)
conn.commit()

In [291]:
Accounts_Country_table = ("""
CREATE TABLE IF NOT EXISTS AccountsCountry (
  Country_Code VARCHAR(6) PRIMARY KEY,
  Long_Name VARCHAR(100),
  Short_Name VARCHAR(100),
  Table_Name VARCHAR(100),
  Currency_Unit VARCHAR(100)
)
""")

In [292]:
cur.execute(Accounts_Country_table)
conn.commit()

In [459]:
Accounts_Data_table1 = ("DROP TABLE IF EXISTS AccountsData")
cur.execute(Accounts_Data_table1)
conn.commit()

In [460]:
Accounts_Data_Table = ("""CREATE TABLE IF NOT EXISTS AccountsData (

    Country_Name VARCHAR(50),
    Country_Code VARCHAR(100),
    Indicator_Name VARCHAR(100),
    Indicator_Code VARCHAR(100),
    Year_1995 FLOAT,
    Year_2000 FLOAT,
    Year_2005 FLOAT,
    Year_2010 FLOAT,
    Year_2015 FLOAT
    
)""")

In [463]:
cur.execute(Accounts_Data_Table)
conn.commit()

In [453]:
Accounts_Series_table1 = ("DROP TABLE IF EXISTS AccountsSeries")
cur.execute(Accounts_Series_table1)
conn.commit()

In [454]:
Accounts_Series_table = ("""CREATE TABLE IF NOT EXISTS AccountsSeries (

    Series_Code VARCHAR(50),
    Topic VARCHAR(50),
    Indicator_Name VARCHAR(200),
    Short_Defination VARCHAR(1000)
    
)""")
cur.execute(Accounts_Series_table)
conn.commit()

In [296]:
Accounts_Country_table_Insert = ("""INSERT INTO AccountsCountry (Country_Code, Long_Name, Short_Name, Table_Name, Currency_Unit)\
VALUES (%s, %s, %s, %s, %s)
""")


In [297]:
for i, row in AccountsCountry_clean.iterrows():
    cur.execute(Accounts_Country_table_Insert, list(row))

In [298]:
conn.commit()

In [464]:
Accounts_Data_table_Insert = ("""INSERT INTO AccountsData (Country_Name, Country_Code, Indicator_Name,
    Indicator_Code, Year_1995, Year_2000,Year_2005,
    Year_2010,
    Year_2015 )
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s )
""")

In [465]:
for i, row in AccountsData.iterrows():
    cur.execute(Accounts_Data_table_Insert, list(row))

In [466]:
conn.commit()

In [455]:
Accounts_Series_table_Insert = ("""INSERT INTO AccountsSeries ( Series_Code, Topic, Indicator_Name,
   Short_Defination)
VALUES (%s, %s, %s, %s)
""")

In [456]:
for i, row in AccountsSeries.iterrows():
    cur.execute(Accounts_Series_table_Insert, list(row))

In [458]:
conn.commit()